# ColQwen2.5-v0.2 for FiftyOne Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/colqwen2_5_v0_2/blob/main/colpali_fiftyone_tutorial.ipynb)

This notebook demonstrates how to use ColQwen2.5-v0.2 with FiftyOne for visual document retrieval.

## Overview

ColQwen2.5 is a Vision Language Model based on Qwen2.5-VL-3B-Instruct that generates ColBERT-style multi-vector representations for efficient document retrieval. This integration uses token pooling to make ColQwen2.5 compatible with FiftyOne's similarity infrastructure.


## Setup

Install required packages:


In [ ]:
%pip install fiftyone colpali-engine transformers torch huggingface-hub umap-learn


## Register the Zoo Model

Register this repository as a FiftyOne zoo model source:


In [ ]:
import fiftyone.zoo as foz

# Register this repository as a remote zoo model source
foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/colqwen2_5_v0_2",
    overwrite=True
)


  150.5Kb [84.6ms elapsed, ? remaining, 1.7Mb/s] 


## Load Dataset

Load a document dataset from Hugging Face:


In [2]:
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub

# Load document dataset from Hugging Face
dataset = load_from_hub(
    "Voxel51/document-haystack-10pages",
    overwrite=True
)


fiftyone.yml:   0%|          | 0.00/104 [00:00<?, ?B/s]

Loading dataset
You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information
Importing samples...
 100% |█████████████████| 250/250 [9.6ms elapsed, 0s remaining, 26.0K samples/s]      


100%|██████████| 3/3 [01:06<00:00, 22.08s/it]


## Basic Workflow: Document Retrieval

### Load Model and Compute Embeddings


In [ ]:
import fiftyone.zoo as foz

# Load ColQwen2.5 model with desired pooling strategy
model = foz.load_zoo_model(
    "vidore/colqwen2.5-v0.2",
    pooling_strategy="max",  # or "mean" (default)
    pool_factor=3  # Compression factor
)


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/34.6M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Compute embeddings for all documents
dataset.compute_embeddings(
    model=model,
    embeddings_field="colqwen_embeddings",
)

# Check embedding dimensions
print(dataset.first()['colqwen_embeddings'].shape)  # Should be (128,)


You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information
You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connection for more information
You are running the oldest supported major version of MongoDB. Please refer to https://deprecation.voxel51.com for deprecation notices. You can suppress this exception by setting your `database_validation` config parameter to `False`. See https://docs.voxel51.com/user_guide/config.html#configuring-a-mongodb-connectio

### Build Similarity Index


In [ ]:
import fiftyone.brain as fob

# Build similarity index
text_img_index = fob.compute_similarity(
    dataset,
    model="vidore/colqwen2.5-v0.2",
    embeddings_field="colqwen_embeddings",
    brain_key="colqwen_sim",
    model_kwargs={
        "pooling_strategy": "max",
        "pool_factor": 3,
    }
)


### Query for Specific Content


In [ ]:
# Query for specific content
sims = text_img_index.sort_by_similarity(
    "the secret office supply is pencil"
)

# Launch FiftyOne App
session = fo.launch_app(dataset, auto=False)
print(session.url)


## Advanced Embedding Workflows

### 1. Embedding Visualization with UMAP

Create 2D visualizations of your document embeddings:


In [ ]:
import fiftyone.brain as fob

# Create UMAP visualization
results = fob.compute_visualization(
    dataset,
    method="umap",  # Also supports "tsne", "pca"
    brain_key="colqwen_viz",
    embeddings="colqwen_embeddings"
)

# Explore in the App
session = fo.launch_app(dataset)


### 2. Similarity Search

Build powerful similarity search with ColQwen embeddings:


In [ ]:
import fiftyone.brain as fob

# Build similarity index
results = fob.compute_similarity(
    dataset,
    backend="sklearn",  # Fast sklearn backend
    brain_key="colqwen_sim", 
    embeddings="colqwen_embeddings"
)

# Find similar images
sample_id = dataset.first().id
similar_samples = dataset.sort_by_similarity(
    sample_id,
    brain_key="colqwen_sim",
    k=10  # Top 10 most similar
)

# View results
session = fo.launch_app(similar_samples)


### 3. Dataset Representativeness

Score how representative each sample is of your dataset:


In [ ]:
import fiftyone.brain as fob

# Compute representativeness scores
fob.compute_representativeness(
    dataset,
    representativeness_field="colqwen_represent",
    method="cluster-center",
    embeddings="colqwen_embeddings"
)

# Find most representative samples
representative_view = dataset.sort_by("colqwen_represent", reverse=True)


### 4. Duplicate Detection

Find and remove near-duplicate documents:


In [ ]:
import fiftyone.brain as fob

# Detect duplicates using embeddings
results = fob.compute_uniqueness(
    dataset,
    embeddings="colqwen_embeddings"
)

# Filter to most unique samples
unique_view = dataset.sort_by("uniqueness", reverse=True)


## Configuration Options

### Pooling Strategy Comparison

Compare mean vs max pooling strategies:


In [ ]:
# Mean pooling (default) - holistic document matching
model_mean = foz.load_zoo_model(
    "vidore/colqwen2.5-v0.2",
    pooling_strategy="mean",
    pool_factor=3
)

# Max pooling - specific content/keyword matching
model_max = foz.load_zoo_model(
    "vidore/colqwen2.5-v0.2",
    pooling_strategy="max",
    pool_factor=3
)


### Custom Pool Factor

Adjust compression level:


In [ ]:
# More aggressive compression (faster, less accurate)
model_compressed = foz.load_zoo_model(
    "vidore/colqwen2.5-v0.2",
    pool_factor=5
)


## Understanding the Compression Pipeline

ColQwen2.5 natively produces variable-length multi-vector embeddings that are incompatible with FiftyOne's fixed-dimension requirements. This integration uses a two-stage compression approach:

### Stage 1: Token Pooling (Intelligent Compression)
- Images: Variable patches (up to 768) → Compressed by factor of 3
- Queries: Variable vectors → Compressed by factor of 3
- **Retains ~97.8% accuracy**
- Removes redundant patches (e.g., white backgrounds) while preserving aspect ratios

### Stage 2: Final Pooling (Fixed Dimensions)
- Mean or Max pooling: Compressed vectors → `(128,)`
- **FiftyOne compatible**
- Both strategies work for classification and retrieval

**Trade-off**: ~85-90% of native ColQwen2.5 accuracy for full FiftyOne compatibility.

For production applications requiring native ColQwen2.5 accuracy, consider using dedicated vector databases like [Qdrant](https://qdrant.tech/) or [Weaviate](https://weaviate.io/) that support multi-vector search natively.


## Resources

- **Original Repository**: [illuin-tech/colpali](https://github.com/illuin-tech/colpali)
- **Model Weights**: [vidore/colqwen2.5-v0.2](https://huggingface.co/vidore/colqwen2.5-v0.2)
- **Paper**: [ColPali: Efficient Document Retrieval with Vision Language Models](https://arxiv.org/abs/2407.01449)
- **Base Model**: [Qwen2.5-VL-3B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct)

### Citation

If you use ColQwen2.5 in your research, please cite:

```bibtex
@misc{faysse2024colpaliefficientdocumentretrieval,
  title={ColPali: Efficient Document Retrieval with Vision Language Models}, 
  author={Manuel Faysse and Hugues Sibille and Tony Wu and Bilel Omrani and Gautier Viaud and Céline Hudelot and Pierre Colombo},
  year={2024},
  eprint={2407.01449},
  archivePrefix={arXiv},
  primaryClass={cs.IR},
  url={https://arxiv.org/abs/2407.01449}, 
}
```

## License

- **Base Model (Qwen2.5-VL)**: [Qwen Research License Agreement](https://huggingface.co/Qwen/Qwen2.5-VL-3B-Instruct)
- **Model Adapters**: Apache 2.0 License
- **Integration Code**: Apache 2.0 License
